In [1]:
from dask.distributed import Client
import xarray as xr
import pandas as pd
import numpy as np
import gcsfs
import sgkit
from sgkit_bgen.bgen_reader import unpack_variables

xr.set_options(display_style='text')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:39801 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 134.78 GB


In [2]:
fs = gcsfs.GCSFileSystem()
store = gcsfs.mapping.GCSMap('rs-ukb/prep-data/gt-imputation/ukb_chrXY.zarr', gcs=fs, check=True, create=False)
ds = xr.open_zarr(store)
ds = unpack_variables(ds, dtype='float16')

# Workaround for https://github.com/pydata/xarray/issues/4386
ds['call_genotype_probability_mask'] = ds['call_genotype_probability_mask'].astype(bool)
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) bool dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [3]:
cr = ds.call_genotype_probability_mask.mean(dim='samples').compute()
cr.max()

<xarray.DataArray 'call_genotype_probability_mask' ()>
array(0.)

In [3]:
# see:
# - https://github.com/Nealelab/UK_Biobank_GWAS/blob/master/0.1/00.load_sample_qc_kt.py
df = pd.read_csv('gs://rs-ukb/prep-data/main/ukb_sample_qc.csv', sep='\t')
df.head()

,eid,x22000_0_0,x22007_0_0,x22008_0_0,x22001_0_0,x22021_0_0,x22006_0_0,x22019_0_0,x22027_0_0,x22003_0_0,...,x22005_0_0,x22020_0_0,x22022_0_0,x22023_0_0,x22025_0_0,x22026_0_0,x22024_0_0,x22028_0_0,x22029_0_0,x22030_0_0
0,4219054,38.0,SMP4_0012173A,F10,1.0,0.0,1.0,NaN,NaN,0.189547,...,0.003946,1.0,592.70,671.82,99.523,0.97,24.82,1.0,1.0,1.0
1,4219069,68.0,SMP4_0012961A,G12,1.0,0.0,1.0,NaN,NaN,0.188485,...,0.012330,1.0,357.16,406.66,98.711,0.97,31.27,1.0,1.0,1.0
2,4219076,78.0,SMP4_0016227A,E04,1.0,1.0,1.0,NaN,NaN,0.186575,...,0.007893,1.0,486.65,423.79,98.995,0.98,47.48,1.0,1.0,1.0
3,4219081,69.0,SMP4_0014664A,F09,0.0,0.0,1.0,NaN,NaN,0.187611,...,0.011950,1.0,479.92,187.74,98.730,0.98,39.84,1.0,1.0,1.0
4,4219097,69.0,SMP4_0014570A,B07,1.0,0.0,NaN,NaN,NaN,0.179372,...,0.003884,1.0,860.80,971.89,99.477,0.98,41.11,1.0,1.0,1.0


In [4]:
len(df)

502505

In [15]:
df["x22019_0_0"].fillna('na').value_counts()  # Sex chromosome aneuploidy

na     501854
1.0       651
Name: x22019_0_0, dtype: int64

In [13]:
df["x22020_0_0"].fillna('na').value_counts()  # Used in genetic principal components

1.0    407127
na      95378
Name: x22020_0_0, dtype: int64

In [3]:
import numpy as np
import dask.array as da
from sgkit.typing import ArrayLike
from xarray import Dataset, DataArray
    
def variant_genotype_counts(ds: Dataset) -> DataArray:
    gti = ds['call_genotype_probability'].argmax(dim='genotypes')
    gti = gti.astype('uint8').expand_dims('genotypes', axis=-1)
    gti = gti == da.arange(ds.dims['genotypes'], dtype='uint8')
    return gti.sum(dim='samples', dtype='uint32')

ds['variant_genotype_counts'] = variant_genotype_counts(ds)[:, [1, 0, 2]]
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    variant_genotype_counts         (variants, genotypes) uint32 dask.array<chunksize=(10432, 3), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [4]:
ds['dummy'] = xr.DataArray(da.empty((ds.dims['variants'], ds.dims['samples'], 2)), dims=('variants', 'samples', 'ploidy'))
ds['variant_hwe_p_value'] = sgkit.hardy_weinberg_test(ds, genotype_counts='variant_genotype_counts')['variant_hwe_p_value']
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, ploidy: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, ploidy, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    variant_genotype_counts         (variants, genotypes) uint32 dask.array<chunksize=(10432, 3), meta=np.ndarray>
    dummy                           (variants, samples, ploidy) float64 dask.array<chunksize=(2186, 2896, 2), meta=np.ndarray>
    variant_hwe_p_value             (variants) float64 dask.array<chunksize=(10432,), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [5]:
%%time
# This requires about 4GiB per core
p = ds['variant_hwe_p_value'].compute()
p

KeyboardInterrupt: 

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
